In [1]:
import numpy as np
import pandas as pd
import itertools
from heliolincrr import HeliolincRR
from lsst.rsp import get_tap_service
from tqdm import tqdm

## Helper functions for DP0.3

In [2]:
def query(adql, con=None):
    """
    DB query
    """
    if con is None:
        con = get_tap_service("ssotap")
    return con.search(adql).to_table().to_pandas()

In [3]:
def create_observations(time_ranges,dist_ranges):
    """
    Query DP0.3 data for observations in a time range and distance range.
    """

    q = """
        SELECT ds.ra,ds.dec,ds.midPointMjdTai,ds.diaSourceId,ds.ssObjectId,
        sss.heliocentricX-sss.topocentricX as observerX,
        sss.heliocentricY-sss.topocentricY as observerY,
        sss.heliocentricZ-sss.topocentricZ as observerZ
        FROM dp03_catalogs_1yr.DiaSource AS ds
        INNER JOIN dp03_catalogs_1yr.SSSource AS sss
        ON ds.diaSourceId = sss.diaSourceId
        WHERE ds.midPointMjdTai >= {:f} AND ds.midPointMjdTai <= {:f} 
        AND sss.heliocentricDist > {:f} AND sss.heliocentricDist <= {:f}
        ORDER BY midPointMjdTai,ds.diaSourceId ASC
    """
    
    obs = query(q.format(time_ranges[0],time_ranges[1],dist_ranges[0],dist_ranges[1]))
    obs[['observerX','observerY','observerZ']] = obs[['observerX','observerY','observerZ']].astype(float)
                
    return obs

In [4]:
def linkable_objects(obs,dts,fields,min_len=6,min_nights=3):
    """
    Find the linkable objects in a set of observations given minimum link constraints.
    """
    obj_id_field = fields[0]
    time_field = fields[1]
    min_tracklets = min_len/min_nights
    
    #filter by detection count constraint first
    uobjs, uobj_counts = np.unique(obs[obj_id_field], return_counts=True) #counts for each object
    uobjs = uobjs[np.where(uobj_counts >= min_len)[0]] #objects with enough detections
    fobs = obs[obs[obj_id_field].isin(uobjs)][[obj_id_field,time_field]].sort_values(time_field) #filter observations without enough detections
    fobs_objid = np.array(fobs[obj_id_field])
    fobs_mjd = np.array(fobs[time_field])
    
    lobjs = [] #list of linkable objects

    for o in tqdm(uobjs, smoothing=0):
        #get all observation times for an object
        obool = fobs_objid == o
        mjds = fobs_mjd[obool]
    
        mtt = 0 #init: minimum tracklet time
        mtts = [] #init: minimum tracklet times for each valid tracklet
        ttimes = [] #init: times of final tracklets (list of lists)
        
        #find all future observations within [dt_min, dt_max] for each observation time
        #choose smallest duration tracklet as one instance for counting purposes
        for mjd in mjds[0:-1]:
            if mjd <= mtt:
                continue
            
            d = mjds - mjd
            mtt_candidates = mjds[(d >= dts[0]) & (d <= dts[1])]
            if len(mtt_candidates) > 0:
                mtt_candidate = np.min(mtt_candidates)
                mtt = mtt_candidate
                mtts.append(mtt)
                ttimes.append([mjd,mtt])
            else:
                continue
    
        #filter by tracklet count constraint (and skip the calcs below if too few)
        num_tls = len(mtts)
        if num_tls < min_tracklets:
            continue

        mtts = np.array(mtts)
        num_dets = len(np.unique(ttimes))
        num_nights = np.sum(np.diff(mtts) > 0.5) + 1
    
        #filter by nights of observation constraint
        if num_nights >= min_nights and num_dets >= min_len:
            lobjs.append([o,num_tls,num_dets,num_nights])
        
    linkables = pd.DataFrame(lobjs,columns=(obj_id_field,'tracklets','sources','nights'))
    
    return linkables

In [5]:
def match_links(links,obs,body_id_field):
    """
    For a set of candidate links, find what observations and objects they match.
    """

    #pure/mixed counts
    pure_objs = []
    pure_links = []
    mixed_objs = []
    mixed_links = []

    for i,l in enumerate(tqdm(links)):
        objs = obs.iloc[l][body_id_field].unique()
        if len(objs)==1:
            pure_objs.append(objs[0])
            pure_links.append(i)
        else:
            mixed_objs.append(objs[0])
            mixed_links.append(i)
            
    return pure_objs, pure_links, mixed_objs, mixed_links

## HeliolincRR: Ingest observations

In [6]:
#get observations between two MJDs and between two heliocentric distances

obs_time_interval = [60218.00491,60218.00491+14] #mjd search interval
#obs_range_interval = [0,1e10] #search all sources (NOTE: ~2hr notebook runtime!)
obs_range_interval = [30,1e10] #search subset of sources (for quick demonstration)
tgt_range_interval = [30,1e10]

#sources we're searching
obs = create_observations(obs_time_interval,obs_range_interval)

#target sources we want to find
tgt_obs = create_observations(obs_time_interval,tgt_range_interval)

In [7]:
#assemble radecs, jdates and observer locations for HeliolincRR instantiation
radecs = np.array(obs[['ra','dec']]) #deg
jdates = np.array(obs['midPointMjdTai'] + 2400000.5) #julian dates
obs_locs = np.array(obs[['observerX','observerY','observerZ']]) #AU

In [8]:
#instantiate HelioLinC with observation data
hl = HeliolincRR(radecs,jdates,obs_locs)

## HeliolincRR: Linking

In [9]:
#CPU cores to use for calculations below
cores = 4

In [10]:
#create tracklets

dts = [5.0/1440,240.0/1440] #tracklet duration boundaries in days
dpds = [0.0,0.05] #degrees per day boundaries on observer sky

#HeliolincRR: create tracklets
hl.create_tracklets(dts,dpds)

generating tracklets: 100%|██████████| 27677/27677 [00:01<00:00, 15021.17it/s]


In [11]:
#specify range/range-rate/range-rate-rate hypotheses, reference epochs and then propagate

#range guesses
rs = np.arange(10,101,10) #r (AU)
rds = [0] #rdot (AU/d)
rdds = [0] #rdotdot (AU/d^2)

#generate all range hypothesis combinations
hypos = [l for l in itertools.product(*[rs,rds,rdds])] 

#define epochs as +/- 1 day from mean unique observation time
epochs = [-1,1] + np.mean(hl.utimes)

#HeliolincRR: propagate tracklets to reference epochs
hl.propagate(hypos,epochs,cores)

propagating: 100%|██████████| 10/10 [00:00<00:00, 15.20it/s]


In [12]:
#cluster propagated tracklets

tol = 0.0006 #position tolerance AU
min_len = 6 #minimum link length
min_nights = 3 #minimum number of nights

#HeliolincRR: cluster propagated tracklets
hypo_links = hl.cluster(tol,min_len,min_nights,cores)

clustering: 100%|██████████| 10/10 [00:06<00:00,  1.61it/s]


## HeliolincRR: Aggregate Results

In [13]:
#match the links HeliolincRR found to what we expect to find
ulinks = hl.ulinks(hypo_links)
pure_objs, pure_links, mixed_objs, mixed_links = match_links(ulinks,obs,'ssObjectId')
tgt_linkable_objects = linkable_objects(tgt_obs,dts,['ssObjectId','midPointMjdTai'])
pure_matches = set(pure_objs) & set(tgt_linkable_objects['ssObjectId'])

100%|██████████| 2374/2374 [00:00<00:00, 11442.57it/s]


In [14]:
s = """Linkable targeted objects: {:d} 
Unique links found across all hypothesis: {:d} 
Unique targeted objects in pure links: {:d} ({:.2f}%)
Mixed links: {:d} ({:.2f}% of unique) """
print(s.format(len(tgt_linkable_objects),len(ulinks),len(pure_matches),100*len(pure_matches)/len(tgt_linkable_objects),len(mixed_objs),100*len(mixed_objs)/len(ulinks)))

Linkable targeted objects: 1914 
Unique links found across all hypothesis: 3037 
Unique targeted objects in pure links: 1910 (99.79%)
Mixed links: 42 (1.38% of unique) 


## HeliolincRR: Inspect an individual link

In [15]:
#each 'ulinks' index contains the indicies of a link HeliolincRR found in the
#observations ('obs') data. ssObjectId should be the same for a pure link.

obs.iloc[ulinks[0]] #the first candidate link HeliolincRR found

,ra,dec,midPointMjdTai,diaSourceId,ssObjectId,observerX,observerY,observerZ
5770,336.203964,-18.673650,60224.02458,5404968455436134042,-3799045400204369929,0.973431,0.208933,0.090550
5774,336.203970,-18.673658,60224.02503,2982152113804713967,-3799045400204369929,0.973429,0.208940,0.090553
6194,336.203482,-18.673724,60224.04943,-6383169764155035792,-3799045400204369929,0.973331,0.209318,0.090715
7970,336.183064,-18.677317,60225.20953,1047390441990126389,-3799045400204369929,0.968268,0.227073,0.098396
8075,336.182686,-18.677380,60225.22977,509706604734492813,-3799045400204369929,0.968175,0.227386,0.098530
16416,336.133561,-18.685311,60228.14658,2323270922036566148,-3799045400204369929,0.953713,0.271507,0.117664
16641,336.133156,-18.685355,60228.17134,1924125803135136329,-3799045400204369929,0.953579,0.271885,0.117825
